# Introduction

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)


import pandas as pd 


drive.mount(ROOT)
%cd '{REPO_PATH}'

In [1]:
repo_name = 'politicalbias'
GIT_REPO = 'https://github.com/thekhan314/politicalbias.git'


from os.path import join
from google.colab import drive

BASE = 'My Drive/Colab Notebooks/'
ROOT = '/content/drive/'
PROJECT_PATH = join(ROOT,BASE)
REPO_PATH = join(PROJECT_PATH,repo_name)

drive.mount(ROOT)
%cd '{REPO_PATH}'

!pip install textdistance

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Colab Notebooks/politicalbias


# Imports

In [2]:

from datashop_classes import *

import pandas as pd
import numpy as np
import scipy as sp
import json

import random
random.seed(11)
import math
from matplotlib import cycler
import matplotlib.pyplot as plt
#import textdistance
import matplotlib as mpl


import os
import glob

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score

import nltk
from nltk import word_tokenize
nltk.download('punkt')
from sklearn.feature_extraction import text as sk_text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

df_adf = pd.read_csv('data/adfontes_political_bias.csv')

df_adf.head()
df_adf.info()
df_adf.describe()


def percentile_bin(score,left_percentile,right_percentile):
    if score <= left_percentile:
        return 'left'
    elif score >= right_percentile:
        return 'right'
    else:
        return "center"

left_perc = np.percentile(df_adf['article_bias'],25)
right_perc= np.percentile(df_adf['article_bias'],75)

df_adf['article_label'] = df_adf['article_bias'].apply(percentile_bin,args=(left_perc, right_perc))


#df_adf['article_bias'].hist()
#df_adf['label'].hist()

NameError: ignored

## IBC Data

Pulled from 
Columns:


In [ ]:
df_ibc = pd.read_csv('data/IBC_data_2_labels.csv')
df_ibc.info()
#df_ibc['Label'].hist()

## Political Bias (Antaripa)

In [ ]:
df_ant = pd.read_csv('data/political_bias_summarized_binary_levels.csv')
df_ant.head()
df_ant.info()

In [ ]:
# Fit a baseline model, pull weights, cluster somehow and find most common hate words

# Adfontes

https://medium.com/analytics-vidhya/mlforsocial-predicting-media-bias-8e94d03befb5

## Load & Process Data

In [28]:
df_adf = pd.read_csv('data/adfontes_political_bias.csv')



def percentile_bin(score,left_percentile,right_percentile):
    if score <= left_percentile:
        return 'left'
    elif score >= right_percentile:
        return 'right'
    else:
        return "center"

left_perc = np.percentile(df_adf['article_bias'],25)
right_perc= np.percentile(df_adf['article_bias'],75)

df_adf['label'] = df_adf['article_bias'].apply(percentile_bin,args=(left_perc, right_perc))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3707 entries, 0 to 3706
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   article_title  3707 non-null   object 
 1   article_bias   3707 non-null   float64
 2   reliability    3707 non-null   float64
 3   article_url    3707 non-null   object 
 4   article_text   3707 non-null   object 
 5   length         3707 non-null   int64  
 6   bias           3707 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 202.9+ KB


In [ ]:
#df_adf.head()
#df_adf.info()
#df_adf.describe()
#df_adf['article_bias'].hist()
#df_adf['label'].hist()

## Overall Word Counts

In [62]:



all_counts = word_counts(df_adf,'article_text', show_chart=False, tokenizer = tokenize,vocab = vocab)
#all_counts.hist(bins = 100)

occur_once = all_counts[all_counts == 1]
#plot_wordcounts(occur_once.sort_values(ascending=True))

high_counts = all_counts[all_counts > 1]

#print(len(high_counts) + len(occur_once),len(all_counts))

vocab = list(high_counts.index)

In [ ]:
stemmer = PorterStemmer()
def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    #tokens = re.split("[^a-zA-Z]*", tweet.lower())
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens
    
all_counts = word_counts(df_adf,'article_text', show_chart=False, tokenizer = tokenize,vocab = vocab)


## Left vs Right counts


In [32]:
df_left = df_adf[df_adf['article_bias'] < left_percentile]
df_right = df_adf[df_adf['article_bias'] > right_percentile]

df_left.info()
df_right.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 919 entries, 0 to 3706
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   article_title  919 non-null    object 
 1   article_bias   919 non-null    float64
 2   reliability    919 non-null    float64
 3   article_url    919 non-null    object 
 4   article_text   919 non-null    object 
 5   length         919 non-null    int64  
 6   bias           919 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 57.4+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 927 entries, 42 to 3692
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   article_title  927 non-null    object 
 1   article_bias   927 non-null    float64
 2   reliability    927 non-null    float64
 3   article_url    927 non-null    object 
 4   article_text   927 non-null    object 
 5   length         92

In [ ]:
left_counts = word_counts(df_left,'article_text')
len(left_counts)

In [ ]:
right_counts = word_counts(df_right,'article_text')
len(right_counts)

In [ ]:
compare_vocabs(left_counts,right_counts)

In [ ]:
compare_vocabs(right_counts,left_counts)

In [21]:
#more context around word occurences? a function that pulls context 
#LDA? 

## Word counts overall

# Modeling 

## Linear Regression

In [50]:
df_adf_1 = df_adf[['article_text','label']]
df_tmod = TopicModel(df_adf_1,'article_text',stop_words='english', n_topics=30,desparsify=True)
print(len(df_tmod.data))
print(df_tmod.data['mod_number'].isnull().sum())
print(df_tmod.data.tail(10))

from sklearn.preprocessing import OneHotEncoder

topic_encoder = OneHotEncoder(sparse=False)
df_encoded = pd.DataFrame(topic_encoder.fit_transform(df_tmod.data[['mod_number']]), columns=topic_encoder.get_feature_names())

df_x = pd.concat([df_tmod.data,df_encoded],axis =1)




KeyboardInterrupt: ignored

In [ ]:
X = df_x.drop(['article_text','mod_number'],axis=1)
y = df_x['article_bias']

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.33, 
    random_state=42)

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

lm = linear_model.LinearRegression()

model = lm.fit(X_train,y_train)
predictions = lm.predict(X_test)

print(mean_squared_error(y_test,predictions))


1.2562899857211965e+24


# Multinomial Naive Bayes

In [18]:

df_adf_2 = df_adf[['article_text','article_label']]
df_tmod = TopicModel(df_adf_2,'article_text',stop_words='english', n_topics=30,desparsify=True)
print(df_tmod.data.shape)


from sklearn.preprocessing import OneHotEncoder

topic_encoder = OneHotEncoder(sparse=False)
df_encoded = pd.DataFrame(topic_encoder.fit_transform(df_tmod.data[['mod_number']]), columns=topic_encoder.get_feature_names())

df_x = pd.concat([df_tmod.data,df_encoded],axis =1)



/content/drive/My Drive/Colab Notebooks/politicalbias/datashop_classes.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['mod_number'] = self.topic_matrix.argmax(axis=1)


(3707, 58001)


In [ ]:
df_tmod

In [64]:
X = df_x.drop(['article_label','article_text'],axis=1)
y = df_x['article_label']

ref = Refinery(X,y)

In [73]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,f1_score
from nltk.stem.porter import *

stemmer = PorterStemmer()
def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    #tokens = re.split("[^a-zA-Z]*", tweet.lower())
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens

mnb = Pipeline([
        ('cvect',CountVectorizer(
            tokenizer = tokenize,
            vocabulary = vocab)
        ),
        ('mnb',MultinomialNB())
         ])

cross_val = cross_validate(
            mnb,X,y,scoring='f1_weighted',cv=6)



/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Found input variables with inconsistent numbers of samples: [58029, 3089]

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Found input variables with inconsistent numbers of samples: [58029, 3090]

  FitFailedWarning)


In [72]:
sorted(sklearn.metrics.SCORERS.keys())

NameError: ignored

In [27]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.33, 
    random_state=42)

mnb = MultinomialNB()

model = mnb.fit(X_train,y_train)
predictions = lm.predict(X_test)

print(f1_score(y_test,predictions,average='weighted'))

0.6608261145187527


In [9]:
y.head()

,label,label
0,left,0
1,center,0
2,center,0
3,center,0
4,center,0


In [51]:
names = df_tmod.vect_object.get_feature_names()

In [53]:
len(X.columns)

58029